In [1]:
# Dependencies
import pandas as pd
import tensorflow as tf
import boto3
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC


health_df = pd.read_csv('Resources/2015.csv')



In [2]:
health_df = health_df.dropna()

health_df

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENUM,...,_PAREC1,_PASTAE1,_LMTACT1,_LMTWRK1,_LMTSCL1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3


In [3]:
health_df.columns.to_list()

['_STATE',
 'FMONTH',
 'IDATE',
 'IMONTH',
 'IDAY',
 'IYEAR',
 'DISPCODE',
 'SEQNO',
 '_PSU',
 'CTELENUM',
 'PVTRESD1',
 'COLGHOUS',
 'STATERES',
 'CELLFON3',
 'LADULT',
 'NUMADULT',
 'NUMMEN',
 'NUMWOMEN',
 'CTELNUM1',
 'CELLFON2',
 'CADULT',
 'PVTRESD2',
 'CCLGHOUS',
 'CSTATE',
 'LANDLINE',
 'HHADULT',
 'GENHLTH',
 'PHYSHLTH',
 'MENTHLTH',
 'POORHLTH',
 'HLTHPLN1',
 'PERSDOC2',
 'MEDCOST',
 'CHECKUP1',
 'BPHIGH4',
 'BPMEDS',
 'BLOODCHO',
 'CHOLCHK',
 'TOLDHI2',
 'CVDINFR4',
 'CVDCRHD4',
 'CVDSTRK3',
 'ASTHMA3',
 'ASTHNOW',
 'CHCSCNCR',
 'CHCOCNCR',
 'CHCCOPD1',
 'HAVARTH3',
 'ADDEPEV2',
 'CHCKIDNY',
 'DIABETE3',
 'DIABAGE2',
 'SEX',
 'MARITAL',
 'EDUCA',
 'RENTHOM1',
 'NUMHHOL2',
 'NUMPHON2',
 'CPDEMO1',
 'VETERAN3',
 'EMPLOY1',
 'CHILDREN',
 'INCOME2',
 'INTERNET',
 'WEIGHT2',
 'HEIGHT3',
 'PREGNANT',
 'QLACTLM2',
 'USEEQUIP',
 'BLIND',
 'DECIDE',
 'DIFFWALK',
 'DIFFDRES',
 'DIFFALON',
 'SMOKE100',
 'SMOKDAY2',
 'STOPSMK2',
 'LASTSMK2',
 'USENOW3',
 'ALCDAY5',
 'AVEDRNK2',
 'DRNK3GE

In [4]:
# Extract data we care about
clean_health_df = health_df[[
                                   
                                   '_MICHD', # Respondents that have ever reported having coronary heart disease (CHD) or myocardial infarction (MI)
                                   '_RFHYPE5', # Adults who have been told they have high blood pressure by a doctor, nurse, or other health professional
                                   'TOLDHI2', # Have you EVER been told by a doctor, nurse or other health professional that your blood cholesterol is high?
                                   '_BMI5', # Body Mass Index (BMI)
                                   'SMOKE100', # Have you smoked at least 100 cigarettes in your entire life?
                                   'CVDSTRK3', # (Ever told) you had a stroke.
                                   'GENHLTH', # Would you say that in general your health is:
                                   'PHYSHLTH', # Now thinking about your physical health, which includes physical illness and injury, for how many days during the past 30 days was your physical health not good?
                                   'SEX', # Indicate sex of respondent
                                   '_AGEG5YR', # Fourteen-level age category
                                    'MENTHLTH' # Now thinking about your mental health, which includes stress, depression, and problems with emotions, for how many days during the past 30 days was your mental health not good?
                                   ]]

print(clean_health_df.shape)
clean_health_df.head(20)

(0, 11)


,_MICHD,_RFHYPE5,TOLDHI2,_BMI5,SMOKE100,CVDSTRK3,GENHLTH,PHYSHLTH,SEX,_AGEG5YR,MENTHLTH


In [5]:
clean_health_df.count()

_MICHD      0
_RFHYPE5    0
TOLDHI2     0
_BMI5       0
SMOKE100    0
CVDSTRK3    0
GENHLTH     0
PHYSHLTH    0
SEX         0
_AGEG5YR    0
MENTHLTH    0
dtype: int64

In [6]:
clean_health_df.isna().sum()

_MICHD      0.0
_RFHYPE5    0.0
TOLDHI2     0.0
_BMI5       0.0
SMOKE100    0.0
CVDSTRK3    0.0
GENHLTH     0.0
PHYSHLTH    0.0
SEX         0.0
_AGEG5YR    0.0
MENTHLTH    0.0
dtype: float64

In [7]:
renamed_df = clean_health_df.rename(columns = {'_MICHD' :'Heart Disease', '_RFHYPE5': 'High Blood Pressure', 'TOLDHI2': 'High Cholestoral',
                                   '_BMI5': 'BMI','SMOKE100': 'Smoked > 100 Cig','CVDSTRK3': 'Had Stroke','GENHLTH': 'General Health','PHYSHLTH':'Physical Health',
                                   'SEX':'SEX', '_AGE5YR': 'AGE RANGE','MENTHLTH': 'MENTAL HEALTH'})

renamed_df.head()

,Heart Disease,High Blood Pressure,High Cholestoral,BMI,Smoked > 100 Cig,Had Stroke,General Health,Physical Health,SEX,_AGEG5YR,MENTAL HEALTH


In [8]:
pip install psycopg2-binary


Note: you may need to restart the kernel to use updated packages.


In [9]:
import psycopg2 as ps

import os

In [10]:
# PG ADMIN CONNECTION

conn = ps.connect(
    host=os.environ["healthcare.c4gamqekwnwh.us-west-1.rds.amazonaws.com"],
    port=os.environ["5432"],
    dbname=os.environ["postgres"],
    user=os.environ["postgres"],
    password=os.environ["F1mir#spec"])

KeyError: 'healthcare.c4gamqekwnwh.us-west-1.rds.amazonaws.com'

In [ ]:
pip install sshtunnel

In [ ]:
from sshtunnel import SSHTunnelForwarder

tunnel = SSHTunnelForwarder(
    ('remote_public_port', 22),
    ssh_username='ubuntu',
    ssh_pkey=mypkey,
    remote_bind_address=('localhost', 5432))
tunnel.start()

conn = psycopg2.connect(
    dbname='healthcare.c4gamqekwnwh.us-west-1.rds.amazonaws.com',
    user='postgres',
    password='F1mir#spec',
    host='127.0.0.1',
    port=tunnel.local_bind_port)

In [ ]:
# from sshtunnel import SSHTunnelForwarder
# # mypkey = amiller.from_private_key_file(path_to_secret_key)
# tunnel = SSHTunnelForwarder(
#     ('remote_public_port', 22),
#     ssh_username='amiller',
#     ssh_pkey=mypkey,
#     remote_bind_address=('localhost', 5432))
# tunnel.start()

conn = ps.connect(
    dbname='healthcare.c4gamqekwnwh.us-west-1.rds.amazonaws.com',
    user='postgres',
    password='F1mir#spec',
    host='127.0.0.1')

In [ ]:
# Import preprocessed (reduced, encoded, etc) data
health_df = pd.read_sql_query("select * from <table>", con=conn)
health_df.head(10)

In [12]:
import psycopg2
engine = psycopg2.connect(
    database="healthcare",
    user="postgres",
    password="F1mir#spec",
    host="healthcare.c4gamqekwnwh.us-west-1.rds.amazonaws.com",
    port='5432'
)

OperationalError: connection to server at "healthcare.c4gamqekwnwh.us-west-1.rds.amazonaws.com" (50.18.184.44), port 5432 failed: FATAL:  database "healthcare" does not exist
